In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # For evaluating accuracy
from tqdm import tqdm
import itertools

# Load the data using pandas
file_path = 'cluster_new_raw_test.csv'
df = pd.read_csv(file_path)

# Drop unnecessary column
df.drop(columns=['id'], inplace=True)

# Show first few rows
df.head()

# Check the distribution of target variable
y = df['Cluster']
y.value_counts()

# Define target column and features
target_column = 'Cluster'
X = df.drop(columns=[target_column])
y = df[target_column]

# Apply standardization (scaling) to the features
scaler = StandardScaler()  # Create the StandardScaler object
X = scaler.fit_transform(X)  # Apply scaling

# Convert to Torch tensors (necessary for GPU-based operations)
X = torch.tensor(X, dtype=torch.float32).cuda()  # Move the features to GPU
y = torch.tensor(y.values, dtype=torch.long).cuda()  # Move target to GPU

# Split the data into train and test (using sklearn)
X_train, X_test, y_train, y_test = train_test_split(X.cpu().numpy(), y.cpu().numpy(), test_size=0.25, random_state=42)

# Convert back to Torch tensors after train-test split
X_train = torch.tensor(X_train, dtype=torch.float32).cuda()
X_test = torch.tensor(X_test, dtype=torch.float32).cuda()
y_train = torch.tensor(y_train, dtype=torch.long).cuda()
y_test = torch.tensor(y_test, dtype=torch.long).cuda()

# Define a simple linear model similar to SGDClassifier
class SGDModel(nn.Module):
    def __init__(self, input_dim):
        super(SGDModel, self).__init__()
        self.fc = nn.Linear(input_dim, 2)  # Binary classification

    def forward(self, x):
        return self.fc(x)

# Function for training and evaluating the model
def train_and_evaluate_model(learning_rate, num_epochs):
    # Initialize the model
    model = SGDModel(X_train.shape[1]).cuda()

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()  # CrossEntropy for classification
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Training Epochs", unit="epoch"):
        model.train()
        optimizer.zero_grad()  # Clear gradients
        outputs = model(X_train)  # Forward pass
        loss = criterion(outputs, y_train)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    # Evaluate the model
    model.eval()  # Switch to evaluation mode
    with torch.no_grad():
        y_pred = model(X_test)  # Make predictions
        _, predicted = torch.max(y_pred, 1)  # Get predicted class labels

    # Evaluate accuracy
    accuracy = accuracy_score(y_test.cpu().numpy(), predicted.cpu().numpy())  # Convert back to NumPy for sklearn
    return accuracy

# Hyperparameter grid for grid search
learning_rates = [0.001, 0.01, 0.1]
num_epochs_list = [50, 100, 150]

# Create a cartesian product of all hyperparameter combinations
param_combinations = list(itertools.product(learning_rates, num_epochs_list))

# Initialize variables to track the best hyperparameters
best_accuracy = 0
best_params = None

# Perform grid search
for lr, epochs in param_combinations:
    print(f"Training with learning_rate={lr} and num_epochs={epochs}")
    accuracy = train_and_evaluate_model(lr, epochs)
    print(f"Accuracy: {accuracy:.4f}")

    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = (lr, epochs)

# Output the best hyperparameters
print(f"Best Hyperparameters: Learning Rate={best_params[0]}, Epochs={best_params[1]}")
print(f"Best Model Accuracy: {best_accuracy:.4f}")


Training with learning_rate=0.001 and num_epochs=50


Training Epochs: 100%|██████████| 50/50 [00:00<00:00, 108.45epoch/s]


Accuracy: 0.2483
Training with learning_rate=0.001 and num_epochs=100


Training Epochs: 100%|██████████| 100/100 [00:00<00:00, 546.67epoch/s]


Accuracy: 0.3835
Training with learning_rate=0.001 and num_epochs=150


...

Accuracy: 0.9065
Training with learning_rate=0.1 and num_epochs=100


Training Epochs: 100%|██████████| 100/100 [00:00<00:00, 557.50epoch/s]


Accuracy: 0.9057
Training with learning_rate=0.1 and num_epochs=150


Training Epochs: 100%|██████████| 150/150 [00:00<00:00, 198.30epoch/s]


Accuracy: 0.9061
Best Hyperparameters: Learning Rate=0.1, Epochs=50
Best Model Accuracy: 0.9065
